In [12]:
import random
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque

class NuclearDQNTransmuter:
    def __init__(self):
        self.element = {"name": "Bismuth", "protons": 83, "neutrons": 126, "mass": 209}
        self.target = {"name": "Gold", "protons": 79, "neutrons": 118, "mass": 197}
        self.energy = 200  # More energy for exploration
        self.actions = 3
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99  # Slower decay for learning
        self.model = self.build_dqn_model()
        self.ensdf_data = self.simulate_ensdf_data()

    def build_dqn_model(self):
        model = tf.keras.Sequential([
            layers.Input(shape=(3,)),
            layers.Dense(32, activation='relu'),  # Bigger network
            layers.Dense(32, activation='relu'),
            layers.Dense(self.actions, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
        return model

    def simulate_ensdf_data(self):
        print("Simulating ENSDF data based on established research...")
        return {
            (83, 126): {"stable": True},
            (83, 127): {"beta-": 1.0, "to": (84, 126), "half_life": 5.012},
            (83, 128): {"beta-": 1.0, "to": (84, 127), "half_life": 0.000238},
            (84, 126): {"alpha": 1.0, "to": (82, 124), "half_life": 138.376},
            (84, 127): {"alpha": 0.998, "beta-": 0.002, "to": (82, 125), "half_life": 60.55},
            (82, 124): {"stable": True},
            (82, 125): {"stable": True},
            (82, 126): {"stable": True},
            (81, 124): {"beta-": 0.99, "to": (82, 124), "half_life": 36.1},
            (81, 123): {"stable": True},
            (80, 122): {"stable": True},
            (80, 123): {"beta-": 0.95, "to": (81, 123), "half_life": 11.5},
            (79, 118): {"stable": True},
            (79, 119): {"beta-": 0.93, "to": (80, 119), "half_life": 3.14}
        }

    def chant_incantation(self):
        chants = [
            "Newton chants: 'By the order of the cosmos, reveal thy essence!'",
            "Newton intones: 'Let the fluxions of nature bend to my will!'",
            "Newton declares: 'Through fire and force, the secret unfolds!'"
        ]
        print(random.choice(chants))
        self.energy += 5
        time.sleep(1)

    def quantum_tunneling_boost(self, action):
        proton_diff = self.element["protons"] - self.target["protons"]
        neutron_diff = self.element["neutrons"] - self.target["neutrons"]
        distance = abs(proton_diff) + abs(neutron_diff)
        base_boost = 1.0 + 0.3 * np.exp(-distance / 3)  # Up to 30% boost
        # Bias toward proton reduction if above 79p
        if action in [1, 2] and proton_diff > 0:  # Proton or carbon bombardment
            return min(1.3, base_boost * 1.2)  # Extra 20% push
        return min(1.3, base_boost)

    def neutron_capture(self):
        state = (self.element["protons"], self.element["neutrons"])
        if state in self.ensdf_data and not self.ensdf_data[state].get("stable", False):
            return False
        base_prob = 0.6  # Reduced to limit neutrons
        if random.random() < base_prob * self.quantum_tunneling_boost(0):
            self.element["neutrons"] += 1
            self.element["mass"] += 1
            self.energy -= 10
            print(f"Neutron capture (quantum boost)! New: {self.element['protons']}p, {self.element['neutrons']}n")
            return True
        print("Neutron capture failed!")
        return False

    def proton_bombardment(self):
        base_prob = 0.7
        if self.energy >= 30 and random.random() < base_prob * self.quantum_tunneling_boost(1):
            self.element["protons"] -= 2
            self.element["neutrons"] -= 2
            self.element["mass"] -= 4
            self.energy -= 30
            print(f"Proton -> Alpha decay (quantum boost)! New: {self.element['protons']}p, {self.element['neutrons']}n")
            return True
        print("Proton bombardment failed!")
        return False

    def carbon_bombardment(self):
        base_prob = 0.6
        if self.energy >= 40 and random.random() < base_prob * self.quantum_tunneling_boost(2):
            # Favor 2p, 2n loss when close to target
            proton_diff = self.element["protons"] - self.target["protons"]
            neutron_diff = self.element["neutrons"] - self.target["neutrons"]
            protons_lost = 2 if abs(proton_diff) <= 4 else random.randint(2, 4)
            neutrons_lost = 2 if abs(neutron_diff) <= 4 else random.randint(2, 4)
            self.element["protons"] -= protons_lost
            self.element["neutrons"] -= neutrons_lost
            self.element["mass"] -= (protons_lost + neutrons_lost)
            self.energy -= 40
            print(f"Carbon bombardment (quantum boost)! Lost {protons_lost}p, {neutrons_lost}n. "
                  f"New: {self.element['protons']}p, {self.element['neutrons']}n")
            return True
        print("Carbon bombardment failed!")
        return False

    def apply_natural_decay(self):
        state = (self.element["protons"], self.element["neutrons"])
        if state in self.ensdf_data and "to" in self.ensdf_data[state]:
            for decay_type in ["beta-", "alpha"]:
                prob = self.ensdf_data[state].get(decay_type, 0)
                if prob > 0 and random.random() < prob:
                    next_state = self.ensdf_data[state]["to"]
                    self.element["protons"], self.element["neutrons"] = next_state
                    self.element["mass"] = self.element["protons"] + self.element["neutrons"]
                    print(f"Natural {decay_type} decay to {self.element['protons']}p, {self.element['neutrons']}n")
                    return True
        return False

    def update_element_name(self):
        names = {83: "Bismuth", 84: "Polonium", 82: "Lead", 81: "Thallium", 80: "Mercury", 79: "Gold"}
        self.element["name"] = names.get(self.element["protons"], "Unknown")

    def get_state(self):
        return np.array([self.element["protons"], self.element["neutrons"], self.energy])

    def act(self, state):
        if random.random() < self.epsilon:
            return random.randrange(self.actions)
        q_values = self.model.predict(state.reshape(1, -1), verbose=0)
        return np.argmax(q_values[0])

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([t[0] for t in minibatch])
        next_states = np.array([t[3] for t in minibatch])
        targets = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            target = reward if done else reward + self.gamma * np.max(next_q_values[i])
            targets[i][action] = target
        self.model.fit(states, targets, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def get_reward(self):
        self.apply_natural_decay()
        proton_diff = self.element["protons"] - self.target["protons"]
        neutron_diff = self.element["neutrons"] - self.target["neutrons"]
        distance = abs(proton_diff) + abs(neutron_diff)
        if self.element["protons"] == self.target["protons"] and \
           abs(self.element["mass"] - self.target["mass"]) <= 2:
            return 200  # Stronger reward for success
        # Exponential penalty with proximity bonus
        penalty = -10 * (proton_diff**2 + neutron_diff**2)
        bonus = 50 * np.exp(-distance / 5) if distance < 10 else 0
        return penalty + bonus

    def check_transmutation(self):
        self.update_element_name()
        if self.element["protons"] == self.target["protons"] and \
           abs(self.element["mass"] - self.target["mass"]) <= 2:
            print(f"Success! Transmuted into {self.element['name']} "
                  f"({self.element['protons']}p, {self.element['neutrons']}n)!")
            return True
        else:
            print(f"Current: {self.element['name']} ({self.element['protons']}p, "
                  f"{self.element['neutrons']}n). Gold still beckons!")
            return False

    def run_transmutation(self):
        print(f"Starting with {self.element['name']} (Mass: {self.element['mass']})...")
        for step in range(40):  # More steps
            if self.energy <= 0:
                break
            self.chant_incantation()
            state = self.get_state()
            action = self.act(state)
            reactions = [self.neutron_capture, self.proton_bombardment, self.carbon_bombardment]
            success = reactions[action]()
            reward = self.get_reward()
            next_state = self.get_state()
            done = self.element["protons"] == self.target["protons"] and \
                   abs(self.element["mass"] - self.target["mass"]) <= 2
            self.memory.append((state, action, reward, next_state, done))
            self.replay()
            if self.check_transmutation():
                break
            print(f"Energy remaining: {self.energy}\n")
        if self.element["name"] == "Gold":
            print("Newton exults: 'The work is done! Behold the metal of the Sun!'")
        else:
            print("Newton laments: 'The cosmos resists... further study is required.'")
        return self.element

# Run simulation
transmuter = NuclearDQNTransmuter()
final_element = transmuter.run_transmutation()

# Weight calculation
initial_mass_bismuth = 100.0
molar_mass_bismuth = 208.9804
molar_mass_gold = 196.9666
moles_bismuth = initial_mass_bismuth / molar_mass_bismuth
final_protons = final_element["protons"]
final_neutrons = final_element["neutrons"]
final_mass_number = final_protons + final_neutrons

if final_protons == 79 and abs(final_mass_number - 197) <= 2:
    final_mass_gold = moles_bismuth * molar_mass_gold
    print(f"\nOriginal weight of Bismuth: {initial_mass_bismuth:.2f} g")
    print(f"Final weight of Gold: {final_mass_gold:.2f} g")
else:
    final_mass_approx = moles_bismuth * final_mass_number
    print(f"\nOriginal weight of Bismuth: {initial_mass_bismuth:.2f} g")
    print(f"Final weight of approximate element ({final_protons}p, {final_neutrons}n): {final_mass_approx:.2f} g")

Simulating ENSDF data based on established research...
Starting with Bismuth (Mass: 209)...
Newton declares: 'Through fire and force, the secret unfolds!'
Carbon bombardment (quantum boost)! Lost 2p, 4n. New: 81p, 122n
Current: Thallium (81p, 122n). Gold still beckons!
Energy remaining: 165

Newton intones: 'Let the fluxions of nature bend to my will!'
Carbon bombardment (quantum boost)! Lost 2p, 2n. New: 79p, 120n
Success! Transmuted into Gold (79p, 120n)!
Newton exults: 'The work is done! Behold the metal of the Sun!'

Original weight of Bismuth: 100.00 g
Final weight of Gold: 94.25 g
